In [1]:
# -----------------------------------------------
# EXO 6 – Interactive Visual Analytics with Folium
# -----------------------------------------------

import pandas as pd
import folium
from folium.features import DivIcon
from folium.plugins import MarkerCluster
from math import sin, cos, sqrt, atan2, radians

# Charger le dataset (dataset_part_2.csv depuis Exo 3)
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_2.csv")

# Ajouter une colonne pour la couleur du marker selon le succès
df['marker_color'] = df['Class'].apply(lambda x: 'green' if x==1 else 'red')

# Coordonnées approximatives du NASA Johnson Space Center
nasa_coordinate = [29.5597, -95.0831]

# Créer la carte centrée sur NASA JSC
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# MarkerCluster pour grouper les markers
marker_cluster = MarkerCluster()

# Ajouter un cercle et un marker pour NASA JSC
circle = folium.Circle(
    location=nasa_coordinate,
    radius=1000,
    color='#d35400',
    fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))

marker = folium.map.Marker(
    location=nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>NASA JSC</b></div>'
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

# Ajouter un marker pour chaque site de lancement
launch_sites = df[['LaunchSite', 'Latitude', 'Longitude']].drop_duplicates()

for index, site in launch_sites.iterrows():
    coord = [site['Latitude'], site['Longitude']]
    folium.Circle(
        location=coord,
        radius=1000,
        color='#0000FF',
        fill=True
    ).add_child(folium.Popup(site['LaunchSite'])).add_to(site_map)
    
    folium.map.Marker(
        location=coord,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:#0000FF;"><b>{site["LaunchSite"]}</b></div>'
        )
    ).add_to(site_map)

# Ajouter un marker pour chaque lancement avec succès/échec
for index, record in df.iterrows():
    folium.Marker(
        location=[record['Latitude'], record['Longitude']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
        popup=f"Booster: {record['BoosterVersion']}\nOutcome: {record['Outcome']}"
    ).add_to(marker_cluster)

site_map.add_child(marker_cluster)

# Fonction pour calculer la distance entre deux points
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # km
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

# Exemple : Marquer la côte la plus proche du site de lancement CCAFS SLC 40
launch_site_coord = [28.561857, -80.577366]
coastline_coord = [28.56367, -80.57163]  # Exemple manuel
distance = calculate_distance(launch_site_coord[0], launch_site_coord[1],
                              coastline_coord[0], coastline_coord[1])

# Ajouter marker pour la côte
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12; color:#d35400;"><b>{distance:.2f} KM</b></div>'
    )
)
distance_marker.add_to(site_map)

# Tracer une ligne entre le site de lancement et la côte
lines = folium.PolyLine(locations=[launch_site_coord, coastline_coord], weight=2, color='blue')
site_map.add_child(lines)

# Afficher la carte interactive
site_map
